In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [2]:
df = pd.read_csv('./data/Gooddreadbooks/ratings.csv')

In [3]:
# Задание 5.1

# Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра random state возьмите число 42.
# Сколько объектов теперь находится в обучающей выборке?

train, test = train_test_split(df, test_size=0.2, random_state=42)
train.shape

(785404, 3)

In [4]:
# Задание 5.2
# Запишите количество уникальных книг в переменную n books.
# Сколько в наборе данных уникальных книг?
n_books = df['book_id'].nunique()
n_books

10000

In [5]:
# Задание 5.3
# Запишите количество уникальных пользователей в переменную n users.
# Сколько в наборе данных уникальных пользователей?
n_users = df['user_id'].nunique()
n_users

53424

In [6]:
# В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [7]:
# Сначала мы задаём размерность входного слоя . После этого определяем размер эмбеддинга - в данном случае снижаем размерность до 5.
# Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten ().
# Делаем то же самое для пользователей:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [8]:
# Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:
conc = Concatenate()([book_vec, user_vec])

In [10]:
# Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. Также мы обозначаем для него
# количество нейронов И данные , которые идут на вход.
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [11]:
# Собираем модель - передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:
model2 = Model([user_input, book_input], out)

In [12]:
# Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать
# метод adam и хорошо известную вам среднеквадратичную ошибку:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [13]:
# Теперь будем обучать нашу модель:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5


2023-08-17 21:55:44.529329: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


24544/24544 [==============================] - 37s 2ms/step - loss: 0.7952
Epoch 2/5
24544/24544 [==============================] - 36s 1ms/step - loss: 0.6879
Epoch 3/5
24544/24544 [==============================] - 38s 2ms/step - loss: 0.6577
Epoch 4/5
24544/24544 [==============================] - 43s 2ms/step - loss: 0.6339
Epoch 5/5
24544/24544 [==============================] - 39s 2ms/step - loss: 0.6130


In [14]:
# Примечание. К сожалению, результаты этого алгоритма нельзя зафиксировать стандартным ramdom state, к которому мы привыкли:
# применяемые методы не используют такой параметр. Поэтому мы опустим здесь сравнение результатов, однако посмотрим, как можно
# настроить нейронную сеть.

# Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения.
# Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав
# десять эПох.

In [15]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 37s 1ms/step - loss: 0.6255
Epoch 2/10
24544/24544 [==============================] - 37s 1ms/step - loss: 0.5772
Epoch 3/10
24544/24544 [==============================] - 37s 2ms/step - loss: 0.5557
Epoch 4/10
24544/24544 [==============================] - 36s 1ms/step - loss: 0.5384
Epoch 5/10
24544/24544 [==============================] - 43s 2ms/step - loss: 0.5244
Epoch 6/10
24544/24544 [==============================] - 46s 2ms/step - loss: 0.5127
Epoch 7/10
24544/24544 [==============================] - 38s 2ms/step - loss: 0.5029
Epoch 8/10
24544/24544 [==============================] - 42s 2ms/step - loss: 0.4947
Epoch 9/10
24544/24544 [==============================] - 42s 2ms/step - loss: 0.4875
Epoch 10/10
6136/6136 [==============================] - 2s 358us/step - loss: 0.7820


0.7819629907608032